# RFM Analysis Homework

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The dataset under study contains credit card transactions from the State of Oklahoma. It includes information on purchases made through the purchase card programs administered by the state and higher educational institutions. The purchase card information is updated monthly after the end of every month. As such, July information is added in August, August in September, and so on and so forth.

In a brief, here's the data description:
- Dataset contains around 440k credit card transactions.
- Each row in the dataset refers to a credit card transaction by a cardholder.
- Each cardholder belongs to an agency.
- Each transaction has the following information:
    - cardholder name
    - agency
    - amount
    - transaction date
    - merchant name (vendor)
    - merchant category
    - description of the expense

## 1. Load and clean the dataset.
Hints:  
a. Transaction date and posted date should be in datetime format  
b. Create a "full name" column  
c. Do not include transactions made by agencies (customer name = agency)  
d. Check for outliers (negative amount, refunds?)  
## 2. Perform an RFM analysis and identify key customer segments (assume that the current date is 2014-07-01).   
Hints:   
a. In this case, we expect a one-to-one relationship between the customers and transactions (i.e. every swipe of the credit card is 1 transaction).  
b. Use quartiles  
## 3. Provide some key insights and recommendations to encourage customers to spend more with the credit cards.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load and clean the data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Refocus/Colab Notebooks/Home Assignments/5.8.5 HA For Submission/res_purchase_card_(pcard)_fiscal_year_2014_3pcd-aiuu.csv',
                   parse_dates=['Transaction Date', 'Posted Date'])

In [ ]:
df.info()

In [ ]:
df.head()

## 1. Load and clean the dataset.
  
a. Create a "full name" column  
b. Do not include transactions made by agencies (customer name = agency)  
c. Check for outliers (negative amount, refunds?)

In [ ]:
# full name column
df['full_name'] = df['Cardholder Last Name'] + ', ' + df['Cardholder First Initial']

In [ ]:
# remove agency (Do not include transactions made by agencies (customer name = agency))
df = df[df['Agency Number'].astype(str)!=df['Cardholder Last Name']]

In [ ]:
df.info()

In [ ]:
# remove negative amounts
df = df[df['Amount']>0]

# RFM Analysis

In [ ]:
import datetime

current_date = datetime.datetime(2014, 7, 1)
current_date

### Recency

In [ ]:
# get max transaction date per customer
df_latest_invoice = df.groupby('full_name')['Transaction Date'].max()

df_latest_invoice

In [ ]:
# get number of days between latest invoice date and current date
# .dt.days means that we only get the actual days (not decimal)
df_recency = (current_date - df_latest_invoice).dt.days

df_recency

### Frequency

In [ ]:
# get frequency (count)
df_frequency = df.groupby('full_name')['full_name'].count()

df_frequency

### Monetary

In [ ]:
# get the total amount (sum of all amounts)
df_monetary = df.groupby('full_name')['Amount'].sum()
df_monetary

## RFM Scores

We now merge these tables into a single dataframe. We put each series together in a dictionary and use `pd.DataFrame`

In [ ]:
df_rfm = pd.DataFrame({'recency':df_recency, 'frequency':df_frequency, 'monetary':df_monetary})
df_rfm

#### Calculate RFM scores. Use quantiles (0.25, 0.50, 0.75) to map the scores from 1 to 4

In [ ]:
quantiles = df_rfm.quantile(q=[0.25, 0.5, 0.75])
quantiles

Using the quantiles function, we can get our thresholds for segmentation.

1. For recency score (lower is better):
- Score = 1; quantile between 0-0.25
- Score = 2; quantile between 0.25-0.5
- Score = 3; quantile between 0.5-0.75
- Score = 4; quantile between 0.75 - 1

<br>  
2. For frequency and monetary score (higher is better):
- Score = 1; quantile between 0.75 - 1
- Score = 2; quantile between 0.5-0.75
- Score = 3; quantile between 0.25-0.5
- Score = 4; quantile between 0-0.25

<br>  

#### Recency Score

In [ ]:
# create a dictionary with the recency quantiles
recency_dict = quantiles['recency'].to_dict()
recency_dict

In [ ]:
# create a function to calculate recency score
def recency_score(recency, recency_dict):
    if recency <= recency_dict[0.25]:
        # recency_score = 1 if recency is between 0-0.25 quantile
        return '1'
    elif recency <= recency_dict[0.5]:
        # recency_score = 2 if recency is between 0.25-0.5 quantile
        return '2'
    elif recency <= recency_dict[.75]:
        # recency_score = 3 if recency is between 0.5-0.75 quantile
        return '3'
    else:
        # recency_score = 4 if recency is between 0.75-1 quantile
        return '4'

df_rfm['recency_score'] = df_rfm['recency'].apply(recency_score, args=(recency_dict,))
df_rfm.head()

### Frequency Score

In [ ]:
# create a dictionary with the frequency quantiles
frequency_dict = quantiles['frequency'].to_dict()
frequency_dict

In [ ]:
# create a function to calculate frequency score
def frequency_score(frequency, frequency_dict):
    if frequency <= frequency_dict[0.25]:
        # frequency_score = 4 if frequency is between 0-0.25 quantile
        return '4'
    elif frequency <= frequency_dict[0.5]:
        # frequency_score = 3 if frequency is between 0.25-0.5 quantile
        return '3'
    elif frequency <= frequency_dict[.75]:
        # frequency_score = 2 if frequency is between 0.5-0.75 quantile
        return '2'
    else:
        # frequency_score = 1 if frequency is between 0.75-1 quantile
        return '1'

df_rfm['frequency_score'] = df_rfm['frequency'].apply(frequency_score, args=(frequency_dict,))
df_rfm.head()

### Monetary Score

In [ ]:
# create a dictionary with the monetary quantiles
monetary_dict = quantiles['monetary'].to_dict()
monetary_dict

In [ ]:
# create a function to calculate monetary score
def monetary_score(monetary, monetary_dict):
    if monetary <= monetary_dict[0.25]:
        # monetary_score = 4 if monetary is between 0-0.25 quantile
        return '4'
    elif monetary <= monetary_dict[0.5]:
        # monetary_score = 3 if monetary is between 0.25-0.5 quantile
        return '3'
    elif monetary <= monetary_dict[.75]:
        # monetary_score = 2 if monetary is between 0.5-0.75 quantile
        return '2'
    else:
        # monetary_score = 1 if monetary is between 0.75-1 quantile
        return '1'

df_rfm['monetary_score'] = df_rfm['monetary'].apply(monetary_score, args=(monetary_dict,))
df_rfm.head()

## Concatenate the RFM Scores

In [ ]:
# Concatenate the three created columns earlier
df_rfm['rfm_score'] = df_rfm['recency_score'] + df_rfm['frequency_score'] + df_rfm['monetary_score']
df_rfm.head(10)

#### Provide insights, recommendations, analysis. Use visualizations if necessary.

In [ ]:
print("Best Cardholder")
len(df_rfm[df_rfm['rfm_score']=='111'])

In [ ]:
# store the segment counts in a dictionary
segment_counts = {'best_cardholder (111)': len(df_rfm[df_rfm['rfm_score']=='111']),
                  'loyal_cardholder (X1X)': len(df_rfm[df_rfm['frequency_score']=='1']),
                  'big_spenders (XX1)': len(df_rfm[df_rfm['monetary_score']=='1']),
                  'almost_inactive (311)': len(df_rfm[df_rfm['rfm_score']=='311']),
                  'inactive_good_cardholders (411)': len(df_rfm[df_rfm['rfm_score']=='411']),
                  'inactive_cheap_cardholders (444)': len(df_rfm[df_rfm['rfm_score']=='444'])}

In [ ]:
segment_counts

In [ ]:
# create a dataframe
df_counts = pd.DataFrame(segment_counts.values(), index=segment_counts.keys(), columns=['counts'])
df_counts

In [ ]:
#plotting the counts
fig, ax = plt.subplots(figsize=(12,7))
df_counts.plot(kind='barh', ax=ax, legend=False)
ax.set_title('RFM Card Holder Segments')
ax.set_ylabel('# Card Holder per Segment')

In [ ]:
# get percentages
cardholder_count = df['full_name'].nunique()

df_counts['pct'] = (df_counts['counts']/cardholder_count) * 100
df_counts['pct'] = df_counts['pct'].round().astype(int)
df_counts

# Visualizations

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
df_counts['pct'].plot(kind='barh', ax=ax, legend=False)
ax.set_title('RFM Card Holder Segments')
ax.set_ylabel('% Card Holder per Segment')

# Labels for the graph with % sign
for index, value in enumerate(df_counts['pct']):
    ax.text(value, index, f"{value}%", ha='left', va='center')


### Top 10 Card Holders

In [ ]:
df_rfm[df_rfm['rfm_score']=='111'].sort_values('monetary', ascending=False).head(10)

# Conclusion and Recommendation

In this analysis, we used a dataset containing credit card transactions from the State of Oklahoma. "Inactive" cardholders are individuals who possess a credit card but do not frequently use it for transactions. They may have the card available for occasional use but do not utilize it regularly.


Based on the bar chart, we observed the percentages of cardholder segments. Our best cardholders constitute 11% of our segmentation base, while 13% are inactive.


As recommendations to encourage increased credit card spending for each segment:


1.	Best Cardholders: Offer premium services, exclusive rewards, and personalized offers to enhance their experience.
2.	Loyal Cardholders: Provide loyalty rewards, points, and referral programs to retain their loyalty.
3.	Big Spenders: Target them with promotions on high-value products or services to further incentivize spending.
4.	Almost Inactive: Remind them about the benefits and perks associated with their credit cards. Utilize limited-time promotions to encourage re-engagement.
5.	Inactive Good Cardholders: Offer exclusive incentives, such as cashback or bonus rewards, to entice them to use their credit cards more frequently.
6.	Inactive Cheap Cardholders: Implement cost-effective offers or benefits to motivate increased spending.
By using these customized approaches, our goal is to strengthen the relationship with cardholders and encourage them to use their credit cards more frequently in all segments.
